In [ ]:

import duckdb

# Connect to a new DuckDB file in your working folder
con = duckdb.connect('gsod_ph.db')

# Create gsod_daily table (simplified example)
con.execute("""
CREATE TABLE IF NOT EXISTS gsod_daily (
    station_id VARCHAR,
    station_name VARCHAR,
    date DATE,
    TEMP_C FLOAT,
    DEWP_C FLOAT,
    MAX_C FLOAT,
    MIN_C FLOAT,
    PRCP_mm FLOAT,
    WDSP_ms FLOAT,
    MXSPD_ms FLOAT,
    GUST_ms FLOAT,
    SLP FLOAT
);
""")


from pathlib import Path
import pandas as pd
from tqdm import tqdm

# Folder containing your cleaned Parquet files
parquet_folder = Path('./gsod_ph_cleaned')
stations_ph = pd.read_csv('isd-history-ph.csv', dtype=str)
stations_ph.columns = stations_ph.columns.str.strip()

# Get all files
parquet_files = sorted(parquet_folder.glob('*.parquet'))

for parquet_file in tqdm(parquet_files):
    df = pd.read_parquet(parquet_file)

    # Extract station_id from filename: e.g., "2020_98427099999.parquet"
    station_id = parquet_file.stem.split('_')[1]

    # Add station_id column
    # df['station_id'] = station_id

    # Extract USAF and WBAN
    usaf = station_id[:6]
    wban = station_id[6:]

    # Lookup station metadata
    station_meta = stations_ph[
        (stations_ph['USAF'].str.strip() == usaf) & 
        (stations_ph['WBAN'].str.strip() == wban)
    ].iloc[0]

    df['station_id'] = station_id
    df['station_name'] = station_meta['STATION NAME']

    
    # Select only required columns and rename if needed
    # df_to_insert = df[['station_id', 'DATE', 'TEMP_C', 'DEWP_C', 'MAX_C', 'MIN_C', 
                       # 'PRCP_mm', 'WDSP_ms', 'MXSPD_ms', 'GUST_ms', 'SLP']].copy()
    df_to_insert = df[['station_id', 'station_name', 'DATE', 'TEMP_C', 'DEWP_C', 'MAX_C', 'MIN_C', 
                   'PRCP_mm', 'WDSP_ms', 'MXSPD_ms', 'GUST_ms', 'SLP']].copy()

    df_to_insert.columns = [col.lower() for col in df_to_insert.columns]  # match SQL casing

    # Insert into DuckDB
    con.execute("INSERT INTO gsod_daily SELECT * FROM df_to_insert")


 74%|███████▍  | 2071/2787 [00:27<00:10, 69.19it/s]

In [13]:
con.close()  # In your Python script


In [7]:
# import duckdb
# import pandas as pd

# con = duckdb.connect('gsod-ph.db')

# # Load station metadata
# stations_df = pd.read_csv('isd-history-ph.csv', dtype=str)

# # Combine USAF + WBAN into station_id to match gsod_daily
# stations_df['station_id'] = stations_df['USAF'].str.strip() + stations_df['WBAN'].str.strip()

# # Keep only what you need
# stations_df = stations_df[['station_id', 'STATION NAME', 'CTRY', 'LAT', 'LON', 'ELEV(M)']].rename(columns={
#     'STATIONNAME': 'station_name',
#     'LAT': 'lat',
#     'LON': 'lon',
#     'ELEV(M)': 'elevation_m'
# })

# # Create stations table
# con.execute("""
# CREATE TABLE IF NOT EXISTS stations (
#     station_id VARCHAR PRIMARY KEY,
#     station_name VARCHAR,
#     CTRY VARCHAR,
#     lat FLOAT,
#     lon FLOAT,
#     elevation_m FLOAT
# );
# """)

# # Insert using pandas (DuckDB supports FROM df directly)
# con.execute("INSERT INTO stations SELECT * FROM stations_df")
